In [1]:
import os
from keras.preprocessing import image
import matplotlib.pyplot as plt 
import numpy as np
from keras.utils.np_utils import to_categorical
import random,shutil
from keras.models import Sequential
from keras.layers import Dropout,Conv2D,Flatten,Dense, MaxPooling2D, BatchNormalization
from keras.models import load_model

In [2]:
# creating model:
model = Sequential()

model.add(Conv2D(64 , (3,3) , activation = 'relu' , input_shape = (24, 24, 1)))
model.add(MaxPooling2D((1, 1)))

model.add(Conv2D(64 , (3,3) , activation = 'relu'))
model.add(MaxPooling2D((1, 1)))

model.add(Conv2D(64 , (3,3) , activation = 'relu'))
model.add(MaxPooling2D((1, 1)))

model.add(Flatten())

model.add(Dense(128, activation = 'relu'))

model.add(Dense(2, activation = 'softmax'))

model.summary ()

2022-05-26 11:19:04.965913: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 22, 22, 64)        640       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 22, 22, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 20, 20, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 20, 20, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 18, 18, 64)        36928     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 18, 18, 64)       0

In [ ]:
import cv2
import os
from keras.models import load_model
import numpy as np
import time
import dlib
from pygame import mixer


mixer.init()
print (os.listdir (r'/Users/pavanteja/Desktop/minor-2/code'))
sound = mixer.Sound(r'/Users/pavanteja/Desktop/minor-2/code/Drowsiness detection_alarm.wav')
face = cv2.CascadeClassifier(r'/Users/pavanteja/Desktop/minor-2/code/haarcascade_lefteye_2splits.xml')
assert not face.empty ()
leye = cv2.CascadeClassifier(r'/Users/pavanteja/Desktop/minor-2/code/haarcascade_lefteye_2splits.xml')
assert not leye.empty ()
reye = cv2.CascadeClassifier(r'/Users/pavanteja/Desktop/minor-2/code/haarcascade_righteye_2splits.xml')
assert not reye.empty ()
PREDICTOR_PATH = "shape_predictor_68_face_landmarks.dat"


lbl=['Close','Open']
model.load_weights('/Users/pavanteja/Desktop/minor-2/code/custmodel.h5')
path = os.getcwd()
predictor = dlib.shape_predictor(PREDICTOR_PATH)
detector = dlib.get_frontal_face_detector()
cap = cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_COMPLEX_SMALL
count=0
score=0
thicc=2
rpred=[99]
lpred=[99]

#Extract information about various features pivotal to detect Yawning
def get_landmarks (im):
    rects = detector(im, 1)
    
    if len(rects) > 1:
        return "error"
    if len(rects) == 0:
        return "error"
    return np.matrix([[p.x, p.y] for p in predictor(im, rects[0]).parts()])


def annotate_landmarks (im, landmarks):
    im = im.copy()
    for idx, point in enumerate(landmarks):
        pos= (point [0, 0], point[0, 1])
        cv2.putText (im, str(idx), pos,
                     fontFace=cv2.FONT_HERSHEY_SCRIPT_SIMPLEX,
                     fontScale=0.4,
                     color=(1, 2, 255))
        cv2.circle(im, pos, 3, color=(0, 2, 2))
    return im

def top_lip(landmarks):
    top_lip_pts = []
    for i in range (50,53):
        top_lip_pts.append(landmarks[i])
    for i in range (61,64):
        top_lip_pts.append(landmarks[i])
    top_lip_all_pts = np.squeeze(np.asarray(top_lip_pts))
    top_lip_mean = np.mean(top_lip_pts, axis=0)
    return int(top_lip_mean[:,1])

def bottom_lip(landmarks):
    bottom_lip_pts = []
    for i in range (65,68):
        bottom_lip_pts.append(landmarks[i])
    for i in range (56,59):
        bottom_lip_pts.append(landmarks[i])
    bottom_lip_all_pts = np.squeeze(np.asarray(bottom_lip_pts))
    bottom_lip_mean = np.mean(bottom_lip_pts, axis=0)
    return int(bottom_lip_mean[:,1])

def mouth_open(image):
    landmarks = get_landmarks(image)
    
    if landmarks == "error":
        return image, 0
    
    image_with_landmarks= annotate_landmarks(image, landmarks)
    top_lip_center = top_lip(landmarks)
    bottom_lip_center = bottom_lip(landmarks)
    lip_distance = abs(top_lip_center-bottom_lip_center)
    return image_with_landmarks, lip_distance

yawns = 0
yawn_status = False

#Detect the eyes from ROI and feed it to the classifier
while(True):
    ret, frame = cap.read()
    height,width = frame.shape[:2]
    image_landmarks, lip_distance = mouth_open(frame)
    prev_yawn_status = yawn_status

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face.detectMultiScale(gray,minNeighbors=5,scaleFactor=1.1,minSize=(25,25))
    left_eye = leye.detectMultiScale(gray)
    right_eye = reye.detectMultiScale(gray)

    cv2.rectangle(frame, (0,height-50) , (200,height) , (0,0,0) , thickness=cv2.FILLED )

    for (x,y,w,h) in faces:
        cv2.rectangle(frame, (x,y) , (x+w,y+h) , (100,100,100) , 1 )

    for (x,y,w,h) in right_eye:
        r_eye=frame[y:y+h,x:x+w]
        count=count+1
        r_eye = cv2.cvtColor(r_eye,cv2.COLOR_BGR2GRAY)
        r_eye = cv2.resize(r_eye,(24,24))
        r_eye= r_eye/255
        r_eye= r_eye.reshape(24,24,-1)
        r_eye = np.expand_dims(r_eye,axis=0)
        rpred = model.predict(r_eye)
        rpred = np.argmax (rpred, axis=1)
        if(rpred[0]==1):
            lbl='Open'
        if(rpred[0]==0):
            lbl='Closed'
        break

    for (x,y,w,h) in left_eye:
        l_eye=frame[y:y+h,x:x+w]
        count=count+1
        l_eye = cv2.cvtColor(l_eye,cv2.COLOR_BGR2GRAY)
        l_eye = cv2.resize(l_eye,(24,24))
        l_eye= l_eye/255
        l_eye=l_eye.reshape(24,24,-1)
        l_eye = np.expand_dims(l_eye,axis=0)
        lpred = model.predict(l_eye)
        lpred = np.argmax(lpred, axis=1)
        if(lpred[0]==1):
            lbl='Open'
        if(lpred[0]==0):
            lbl='Closed'
        break

    if(rpred[0]==0 and lpred[0]==0 and lip_distance > 25):
        yawn_status = True
        score=score+1
        cv2.putText(frame,"Driver is Yawning and eyes Closed",(10,height-60), font, 1,(255,255,255),1,cv2.LINE_AA)
        output_text= " Yawn Count: " + str(yawns + 1)
        cv2.putText(frame, output_text, (50,50),font, 1, (0,255,127), 2)
        try:
            sound.play()
            
        except: # isplaying = False
            pass
    
    if(rpred[0]==0 and lpred[0]==0):
        score=score+1
        cv2.putText(frame,"Eyes Closed",(10,height-40), font, 1,(255,255,255),1,cv2.LINE_AA)
    else:# if(rpred[0]==1 or lpred[0]==1):
        yawn_status = False
        score=score-1
        cv2.putText(frame,"Open",(10,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)

    if(score<0):
        score=0
    cv2.putText(frame,'Score:'+str(score),(100,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
    if(score>15):
        #we beep the alarm as person is feeling sleepy
        cv2.imwrite(os.path.join(path,'image.jpg'),frame)
        try:
            sound.play()
            
        except: # isplaying = False
            pass
        
        if(thicc<5):
            thicc= thicc+2
        else:
            thicc=thicc-2
            if(thicc<2):
                thicc=2
        cv2.rectangle(frame,(0,0),(width,height),(0,0,255),thicc)
    cv2.imshow('frame',frame)
    cv2.imshow('Live Landmarks', image_landmarks )
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

pygame 2.1.2 (SDL 2.0.18, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html
['custmodel.h5', 'shape_predictor_68_face_landmarks.dat', '.DS_Store', 'drowsiness detection.py', 'dcr.hdf5', 'Drowsiness detection_alarm.wav', 'haarcascade_lefteye_2splits.xml', 'drowsiness and yawning detection.py', 'model.py', 'haarcascade_righteye_2splits.xml', 'haarcascade_frontalface_alt.xml', '.ipynb_checkpoints', 'Drowsiness and Yawn Detector.ipynb', 'dcr.json', 'dcr.h5', 'image.jpg']


/var/folders/zl/z7y9tfcd7kg54lfk1ccm84140000gn/T/ipykernel_1056/2098997416.py:80: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if landmarks == "error":
